[Source Code Jupyter Notebook](https://github.com/mctosima/belajarpytorch/blob/main/docs/01_HelloWorldnyaDL.ipynb)

# 01. Hello-World Ala Deep Learning

Pada bagian ini kamu diharapkan mampu:
- Memahami _workflow_ training menggunakan PyTorch
- Memahami penggunaan _optimizer_ dan _loss function_
- Memahami proses training sederhana
- Menggunakan model untuk inferensi
- Mengukur performa model secara sederhana

<div align="center">
<img src="https://images.contentstack.io/v3/assets/blt71da4c740e00faaa/blt3e9883f5dfd008f4/603039d9cb67827268e09219/saltbae_pytorch.jpg" width="350"/>
</div>

> Pada materi kali ini kita akan mencoba membuat model kecil-kecilan untuk memprediksi keluaran dari dua buah masukan berupa boolean. Sederhananya, kita akan membuat model yang mampu memprediksi hasil dari XOR tanpa memberitahu model bahwa operasi ini adalah XOR. Model akan belajar sendiri untuk melakukan operasi XOR.

**Tttt.tapi, saya pun tidak tahu apa itu XOR?**

Baiklah, jadi begini. XOR adalah operasi logika yang menghasilkan nilai `True` jika salah satu dari dua buah masukan bernilai `True` dan yang satunya bernilai `False`. Jika kedua buah masukan bernilai `True` atau `False`, maka hasilnya adalah `False`.

**Masih belum mengerti**

Baiklah, berikut ini adalah tabel logika XOR:

<div align="center">
<img src="https://www.researchgate.net/publication/326006336/figure/tbl1/AS:669038808141849@1536522690995/Truth-table-of-the-logical-operator-XOR.png" width="350"/>
</div>

Misalkan, jika diberi sebuah masukan X1 berupa 1 dan X2 berupa 0, maka hasil dari operasi XOR adalah 1. Jika diberi masukan X1 berupa 0 dan X2 berupa 0, maka hasil dari operasi XOR adalah 0. Begitu juga sebaliknya. ***Sama jadi 0, beda jadi 1.***

## Import Library

In [1]:
import torch
import torch.nn as nn

- `import torch` merupakan library utama yang akan kita gunakan. Library ini berisi fungsi-fungsi yang akan kita gunakan untuk membuat model, menghitung loss, dan melakukan training.
- `import torch.nn as nn` merupakan library yang berisi fungsi-fungsi yang akan kita gunakan untuk membuat model.

## Mempersiapkan data masukan

Kita akan membuat dua buah variabel, `x1` dan `x2` dimana masing-masing akan berisi 50 buah nilai boolean. Kita akan menggunakan `torch.rand` untuk menghasilkan nilai random dari 0 sampai 1.

In [2]:
# membuat variabel x berisi nilai boleean berisi 1 atau 0 sebanyak 50 elemen
x1 = torch.randint(0, 2, (50,))
x2 = torch.randint(0, 2, (50,))
print(f"Input X1:\n{x1}")
print(f"Input X2:\n{x2}")

Input X1:
tensor([1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        1, 0])
Input X2:
tensor([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 0])


Selanjutnya kita menggabungkan `x1` dan `x2` menjadi satu buah variabel `x` dengan menggunakan `torch.stack`. Karena `x1` dan `x2` masing-masing berukuran `(50,)`, maka `x` akan berukuran `(2, 50)`.

In [3]:
# menggabungkan variabel x1 dan x2 menjadi satu variabel x
x = torch.stack((x1, x2), dim=1)

Setelah itu, kita akan membuat variabel `y` yang berisi nilai boolean dari hasil operasi XOR dari `x1` dan `x2`. Kita akan menggunakan tanda `^` untuk melakukan operasi XOR. Kamu dapat mencocokkan beberapa contoh masukan dan keluaran sebenarnya (`y`). Misalnya pada indeks-0, `x1` bernilai 1 dan `x2` bernilai 1, maka indeks-0 dari `y` bernilai 0.

> **Catatan:** `y` dalam hal ini disebut juga sebagai `label` atau nama lainnya kerap disebut sebagai `ground truth`

In [4]:
# membuat ground-truth dari input x1 dan x2 yang sudah distack pada x
y = x[:, 0] ^ x[:, 1]
print(f"Ground-truth Y:\n{y}")

Ground-truth Y:
tensor([0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
        0, 0])


Selanjutnya, kita akan membagi dataset `x` dan `y` menjadi `train` dan `test`.

**Mengapa harus dibagi menjadi train dan test?**
- Ketika kita melakukan proses training, label (`y`) akan diberitahu ke model sehingga jawaban sebenarnya akan diketahui model. Namanya juga proses training, tentu model harus tahu jawaban yang benarnya sehingga dia dapat belajar dari kesalahan yang dia buat.
- Oleh karena itu tidak adil jika kita menguji apakah model yang kita buat sudah pinter atau belum dengan bahan belajar yang sudah ada kunci jawabannya
- Maka dari itu kita menyediakan test data sebagai bahan ujian untuk mengetahui seberapa baik model yang kita buat.

In [5]:
train_split = int(0.8*len(x))

In [6]:
# membagi x menjadi train dan testing
x_train = x[:train_split]
x_test = x[train_split:]

# membagi y menjadi train dan testing
y_train = y[:train_split]
y_test = y[train_split:]

In [7]:
# membuat model regresi linear
class XORModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 2)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(2, 1)
        self.sigmoid2 = nn.Sigmoid()
        

    def forward(self, x):
        x = x.float()
        x = self.linear(x)
        x = self.sigmoid(x)
        x = self.linear2(x)
        x = self.sigmoid2(x)
        return x

In [8]:
# create instance
model = XORModel()

# check state_dict
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[-0.2457,  0.6129],
        [ 0.5071,  0.1579]])), ('linear.bias', tensor([-0.2959,  0.6638])), ('linear2.weight', tensor([[0.3175, 0.3664]])), ('linear2.bias', tensor([0.1200]))])


In [9]:
# mencoba inferensi dengan input x
y_pred = model(x_test)
print(f"Input:\n{x_test}")
print(f"Hasil prediksi:\n{y_pred}")
print(f"Ground-truth:\n{y_test}")

Input:
tensor([[1, 1],
        [1, 0],
        [0, 1],
        [1, 1],
        [1, 1],
        [0, 1],
        [0, 0],
        [0, 0],
        [1, 1],
        [0, 0]])
Hasil prediksi:
tensor([[0.6397],
        [0.6263],
        [0.6360],
        [0.6397],
        [0.6397],
        [0.6360],
        [0.6218],
        [0.6218],
        [0.6397],
        [0.6218]], grad_fn=<SigmoidBackward0>)
Ground-truth:
tensor([0, 1, 1, 0, 0, 1, 0, 0, 0, 0])


In [10]:
# mendefinisikan loss function
loss_fn = nn.MSELoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# epoch
epochs = 5000

In [11]:
# training loop
for epoch in range(epochs):
    # training
    y_pred = model(x_train)
    y_train = y_train.float()
    loss = loss_fn(y_pred, y_train.unsqueeze(1))

    # backward
    optimizer.zero_grad()
    
    loss.backward()
    
    
    optimizer.step()
    
    # testing
    with torch.no_grad():
        y_test_pred = model(x_test)
        y_test = y_test.float()
        test_loss = loss_fn(y_test_pred, y_test.unsqueeze(1))

    # print loss only 3 digit after comma. Log every 500 epoch
    if (epoch+1) % 200 == 0:
        print(f"Epoch: {epoch+1}, Loss: {loss.item():.3f}, Test Loss: {test_loss.item():.3f}")

Epoch: 200, Loss: 0.247, Test Loss: 0.259
Epoch: 400, Loss: 0.239, Test Loss: 0.246
Epoch: 600, Loss: 0.193, Test Loss: 0.176
Epoch: 800, Loss: 0.160, Test Loss: 0.141
Epoch: 1000, Loss: 0.139, Test Loss: 0.136
Epoch: 1200, Loss: 0.098, Test Loss: 0.095
Epoch: 1400, Loss: 0.016, Test Loss: 0.016
Epoch: 1600, Loss: 0.008, Test Loss: 0.008
Epoch: 1800, Loss: 0.005, Test Loss: 0.005
Epoch: 2000, Loss: 0.004, Test Loss: 0.004
Epoch: 2200, Loss: 0.003, Test Loss: 0.003
Epoch: 2400, Loss: 0.002, Test Loss: 0.003
Epoch: 2600, Loss: 0.002, Test Loss: 0.002
Epoch: 2800, Loss: 0.002, Test Loss: 0.002
Epoch: 3000, Loss: 0.002, Test Loss: 0.002
Epoch: 3200, Loss: 0.001, Test Loss: 0.001
Epoch: 3400, Loss: 0.001, Test Loss: 0.001
Epoch: 3600, Loss: 0.001, Test Loss: 0.001
Epoch: 3800, Loss: 0.001, Test Loss: 0.001
Epoch: 4000, Loss: 0.001, Test Loss: 0.001
Epoch: 4200, Loss: 0.001, Test Loss: 0.001
Epoch: 4400, Loss: 0.001, Test Loss: 0.001
Epoch: 4600, Loss: 0.001, Test Loss: 0.001
Epoch: 4800, Lo

In [12]:
# mencoba inferensi dengan input yang diberikan pengguna
x_user = torch.tensor([[1, 0]])
y_truth = x_user[:, 0] ^ x_user[:, 1]
y_pred = model(x_user)

print(f"Input:\n{x_user}")
print(f"Hasil prediksi:\n{y_pred}")
print(f"Ground-truth:\n{y_truth}")

Input:
tensor([[1, 0]])
Hasil prediksi:
tensor([[0.9724]], grad_fn=<SigmoidBackward0>)
Ground-truth:
tensor([1])
